## 전체 영양제 리뷰 전처리 시작

In [1]:
import pandas as pd

In [2]:
df_all=pd.read_csv('../datasets/reviews_data/all_product.csv')
df_all[-2:]

product_name review_date  \
90624  푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복  2022.04.06   
90625  푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복  2022.03.27   

                                       review_content  review_star  \
90624  배송 보통이에요 성능 보통이에요 유통기한 보통이에요\n지인 선물을줬는데  좋아했어요          5.0   
90625                 배송 보통이에요 성능 성능이 뛰어나요 유통기한 보통이에요          5.0   

      review_writer  
90624           채슨생  
90625       smcd***

### 불용어 처리 및 한글자 살리기 및 단어 치환, 

#### 전처리 필요한 파일들 가져오기

In [3]:
# 불용어 
df_stop = pd.read_excel('../datasets/dic/stopword_list.xlsx')
df_stop[-2:]

stopword
1177     \r\n
1178       \n

In [4]:
# 한글자 살리기
df_oneword=pd.read_excel('../datasets/dic/one_char_list.xlsx')
df_oneword[-2:]

one_char_keyword
12                굿
13                굳

#### 전처리 시작 
- 1. 형태소 분석 및 토큰화
- 2. 단어 치환
- 3. 불용어 처리


In [5]:
from konlpy.tag import Okt
from collections import Counter

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# df_all['reveiw_replaced'] 열의 각 텍스트에 대해 중복 단어 제거 및 형태소 분석 수행
tokenized_reviews = []

# 필요한 품사 리스트 정의
desired_pos = ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Conjunction']

# 형태소 분석 결과를 담을 리스트 초기화
tokenized_reviews = []

for review_text in df_all['review_content']:
    try:
        # 형태소 분석 수행
        pos_result = okt.pos(review_text, norm=True, stem=True)
        
        # 필요한 조건을 만족하는 단어 선택
        filtered_tokens = []
        for word, pos in pos_result:
            if pos in desired_pos and len(word) > 1:
                filtered_tokens.append(word)
                    
        tokenized_reviews.append(filtered_tokens)  # 각 토큰을 개별 항목으로 유지
    except:
        pass

In [6]:
import pickle

In [7]:
# 형태소 분석 결과를 별도의 pkl 파일로 저장합니다.
# with open('tokenized_reviews.pkl', 'wb') as preprocess_token:
#     pickle.dump(obj=df_all['tokenized_reviews'], file=preprocess_token)
with open('tokenized_reviews.pkl', 'wb') as preprocess_token:
    pickle.dump(obj=tokenized_reviews, file=preprocess_token)

In [8]:
df_all['tokenized_reviews']=tokenized_reviews 

In [9]:
# 데이터프레임 df_all의 행 수 확인
df_all_shape = df_all.shape[0]

# tokenized_reviews 리스트의 길이 확인
tokenized_reviews_length = len(tokenized_reviews)

# 데이터프레임 df_all과 tokenized_reviews의 크기 비교
if df_all_shape == tokenized_reviews_length:
    print("데이터프레임 df_all과 tokenized_reviews의 크기가 일치합니다.")
else:
    print("데이터프레임 df_all과 tokenized_reviews의 크기가 일치하지 않습니다.")
    print(f"df_all의 행 수: {df_all_shape}")
    print(f"tokenized_reviews의 길이: {tokenized_reviews_length}")

데이터프레임 df_all과 tokenized_reviews의 크기가 일치합니다.


In [10]:
# df_all['tokenized_reviews']=tokenized_reviews

In [11]:
# # 단어 치환 함수
# def replace_word(review):
#     for number in range(len(df_replace['before_replacement'])):
#         try: 
#             # 치환 단어만 데이터 치환
#             if replace_list['before_replacement'][number] in review:
#                 review = review.replace(replace_list['before_replacement'][number], replace_list['after_replacement'][number])
#         except: 
#             pass
#     return review

In [12]:
# # 치환된 리뷰 생성
# df_all['review_replaced'] = df_all['tokenized_reviews'].apply(replace_word)

In [13]:
# # 치환된 단어만 출력
# for replaced_review_list in df_all['review_replaced']:
#     for replaced_review in replaced_review_list:
#         replaced_tokens = replaced_review.split()
        
#         # 치환된 단어를 한 줄에 출력
#         for replaced in replaced_tokens:
#             print(f"치환된 단어: {replaced}")

In [14]:
# df_all['review_replaced']

In [15]:
# 불용어 목록을 리스트로 변환
stop_list = df_stop['stopword'].tolist()

# 불용어 처리 함수 정의
def stopword(tokens):
    removed_tokens = [word for word in tokens if word not in stop_list]
    return ' '.join(removed_tokens)

# df_all['origin_reviews'] 열의 각 리스트에 불용어 처리 적용하고 쪼개기
df_all['replaced_review'] = df_all['tokenized_reviews'].apply(stopword)

In [20]:
import pickle

# 불용어 목록을 리스트로 변환
stop_list = df_stop['stopword'].tolist()

# 불용어 사전을 pkl로 저장
with open('../datasets/stopword.pkl', 'wb') as stopword:
    pickle.dump(obj=stop_list, file=stopword)

In [16]:
# # 형태소 분석 결과를 별도의 pkl 파일로 저장합니다.
# with open('replaced_review.pkl', 'wb') as preprocess_stopword:
#     pickle.dump(obj=df_all['replaced_review'], file=preprocess_stopword)

In [17]:
df_all['replaced_review'][2299]

'빨르다 뛰어나다 고지혈증 수치 높다 편이 보조 제로 도움'

In [18]:
type(df_all['replaced_review'][2299])

str

In [19]:
# pd.read_csv('../datasets/reviews_data/all_preprocess.csv')

In [22]:
df_all[22222:22230]

product_name review_date  \
22222  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.06   
22223  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.04   
22224  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.04   
22225  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.03   
22226  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.03   
22227  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.02   
22228  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.02   
22229  종근당건강 아이케어 오메가3 6개월분 비타민A 비타민E 혈행 중성지질 개선 항산화 ...  2018.03.01   

                                    review_content  review_star review_writer  \
22222        배송 보통이에요 맛 보통이에요 유통기한 보통이에요\r\n잘먹고잇슴당          5.0        내알아잘사요   
22223          배송 빨라요 맛 맛있어요 유통기한 기한이 넉넉해요\r\n잘샀어요          5.0        ahm3**   
22224  배송 빨라요 맛 맛있어요 유통기한 기한이 넉넉해요\r\n배송도빠르고 넘좋아요~          5.0      syhr****   
22225        배송 빨라요 맛 맛있어요 유통기한 기한이 넉넉해요\r\n싸고 좋아요          5.0    alli******   
22226      배송 보통이에요 맛 보통이에요 유통기한 보통이에요\r\n건강잘챙겨볼께요          5.0        buly**   
22227                 배송 빨라요 맛 보통이에요 유통기한 기한이 넉넉해요          5.0      dusl****   
22228                 배송 빨라요 맛 보통이에요 유통기한 기한이 넉넉해요          5.0     nahe*****   
22229         배송 빨라요 맛 맛있어요 유통기한 기한이 넉넉해요\r\n잘먹을게요          5.0      simd****   

                                       tokenized_reviews replaced_review  
22222                   [배송, 보통, 보통, 유통, 기한, 보통, 먹다, 잇다]              잇다  
22223               [배송, 빨르다, 맛있다, 유통, 기한, 기한, 넉넉하다, 사다]      빨르다 맛있다 사다  
22224  [배송, 빨르다, 맛있다, 유통, 기한, 기한, 넉넉하다, 배송, 빠르다, 넘다, 좋다]   빨르다 맛있다 넘다 좋다  
22225           [배송, 빨르다, 맛있다, 유통, 기한, 기한, 넉넉하다, 싸다, 좋다]   빨르다 맛있다 싸다 좋다  
22226              [배송, 보통, 보통, 유통, 기한, 보통, 건강, 챙기다, 보다]              건강  
22227                    [배송, 빨르다, 보통, 유통, 기한, 기한, 넉넉하다]             빨르다  
22228                    [배송, 빨르다, 보통, 유통, 기한, 기한, 넉넉하다]             빨르다  
22229               [배송, 빨르다, 맛있다, 유통, 기한, 기한, 넉넉하다, 먹다]         빨르다 맛있다

In [ ]:
# # 리스트 형태의 문서를 텍스트로 변환
# corpus_text = [' '.join(doc) for doc in df_all['replaced_review']]

In [ ]:
# df_all['replaced_review'] = corpus_text

In [ ]:
df_all

In [ ]:
df_all.to_csv('../datasets/reviews_data/all_preprocess.csv')

In [ ]:
df_all.isnull().sum()

In [ ]:
# df_all=pd.read_csv('../datasets/reviews_data/all_preprocess.csv')

In [ ]:
df_all

In [ ]:
# review_star 부정과 긍정 댓글 분리 
condition_positive = df_all['review_star'] > 3  # 긍정
condition_negative = df_all['review_star'] <= 3  # 부정

In [ ]:
df_review_positive = df_all[condition_positive]
df_review_negative = df_all[condition_negative]

In [ ]:
df_review_positive.to_csv('../datasets/reviews_data/all_preprocess_positive.csv')
df_review_negative.to_csv('../datasets/reviews_data/all_preprocess_negative.csv')

In [ ]:
len(df_review_positive),len(df_review_negative)

In [ ]:
# df_review_positive=df_review_positive.drop(columns=['okt_reviews','stopremoved_reviews'])
# df_review_negative=df_review_negative.drop(columns=['okt_reviews','stopremoved_reviews'])

In [ ]:
# df_review_positive.to_csv('../datasets/reviews_data/final_all_preprocess_positive.csv')
# df_review_negative.to_csv('../datasets/reviews_data/final_all_preprocess_negative.csv')

In [ ]:
# df_recovery['mecab_reviews']
df_all['replaced_review']

In [ ]:
# 'mecab_reviews' 열의 모든 텍스트를 하나의 문자열로 결합
#combined_text = ' '.join(df_recovery['mecab_reviews'])
combined_text = ' '.join(df_all['replaced_review'])

In [ ]:
from wordcloud import WordCloud

In [ ]:
from collections import Counter

#### 토큰화 리뷰 빈도 계산

###### 형태소의 빈도를 계산할때 

In [ ]:
from collections import Counter

# 단어 빈도수 카운트
word_counts = Counter(combined_text.split())

In [ ]:
# 한글 폰트 경로 설정 (Windows 환경에 맞게 경로 수정)
font_path = 'C:/Windows/Fonts/malgun.ttf'

# 단어 빈도수 딕셔너리를 WordCloud에 전달
wordcloud = WordCloud(font_path=font_path, background_color='white')
wordcloud.generate_from_frequencies(word_counts)

In [ ]:
wordcloud.words_

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis('off') 
plt.show()

###### 문장을 합쳐서 계산할때 

In [ ]:
# 단어로 쪼개기 
text_split = combined_text.split()

In [ ]:
# counter를 이용해서 중복 여부 count
from collections import Counter
Counter(text_split)

In [ ]:
# 한글 폰트 경로 설정 (Windows 환경에 맞게 경로 수정)
font_path = 'C:/Windows/Fonts/malgun.ttf'

wordCloud = WordCloud(font_path=font_path, background_color='white') #인스턴스화 
wordCloud.generate_from_text(combined_text)

In [ ]:
# count 
wordCloud.words_ 

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(wordCloud)
plt.axis('off') 
plt.show()

In [ ]:
### 단어 치환하는 코드
def second_replace(tokens, replace_dict):
    return [replace_dict.get(word, word) for word in tokens]

In [ ]:
# 대체어
df_replace = pd.read_excel('../datasets/dic/replace_list.xlsx')
df_replace[-2:]

In [ ]:
replace_dict = df_replace.set_index('before_replacement').to_dict()['after_replacement']

In [ ]:
# 2차 단어 치환 적용
df_all['tokenized_reviews'] = df_all['replaced_review'].apply(second_replace, replace_dict=replace_dict)

In [ ]:
# 결과 확인: 처음 5개 레코드의 2차 단어 치환 결과 출력
for i, tokens in enumerate(df_all['tokenized_reviews'][:5]):
    original_review = df_all['review_content'][i]
    replaced_review = ' '.join(tokens)
    print(f"Original Review ({i + 1}): {original_review}")
    print(f"Replaced Review ({i + 1}): {replaced_review}")
    print()